In [ ]:
import random
import cv2
from keras import backend as K
from keras.preprocessing import image
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.compat.v1.logging import INFO, set_verbosity

def load_image(img, image_dir, df, preprocess=True, H=320, W=320):
    """Load and preprocess image."""
    img_path = image_dir + img
    mean, std = get_mean_std_per_batch(img_path, df, H=H, W=W)
    x = image.load_img(img_path, target_size=(H, W))
    if preprocess:
        x -= mean
        x /= std
        x = np.expand_dims(x, axis=0)
    return x

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

from keras.models import load_model


# from tensorflow.keras.applications import DenseNet121
import tensorflow as tf
import tensorflow.keras.layers as L
# import tensorflow.keras.layers as Layers

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
IMAGE_SIZE=[224,224]
EPOCHS = 20
BATCH_SIZE = 64


In [ ]:
#importing dataset 
import pandas as pd 
train_df_main = pd.read_csv('train_df.csv')
# valid_df = pd.read_csv("nih/valid-small.csv")
# test_df = pd.read_csv("nih/test.csv")

train_df_main.drop(['No Finding'], axis = 1, inplace = True)
labels = train_df_main.columns[2:-1]
labels

In [ ]:
from sklearn.model_selection import train_test_split
train_df, discard = train_test_split(train_df_main, test_size = 0.7, random_state = 1993)

train_and_valid_set, test_set = train_test_split(train_df, test_size = 0.2, random_state = 1993)
train_set, valid_set = train_test_split(train_and_valid_set, test_size = 0.2, random_state = 1993)

In [ ]:
#data leakage eg multiple patient overlaps
def check_for_leakage(df1, df2, patient_col):
    df1_patients_unique = set(df1[patient_col].values)
    df2_patients_unique = set(df2[patient_col].values)
    patients_in_both_groups = df1_patients_unique.intersection(df2_patients_unique)
    # leakage contains true if there is patient overlap, otherwise false.
    leakage = len(patients_in_both_groups)>0 
    return leakage

In [ ]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=8, seed=1, target_w = 320, target_h = 320):
   
    print("getting train generator...")
    # normalize images
    image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization= True, 
        shear_range=0.1,
        zoom_range=0.15,
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.05,
        horizontal_flip=True, 
        vertical_flip = False, 
        rescale=1.0/255.0,
        fill_mode = 'reflect')
    
    
    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=None,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            target_size=(target_w,target_h))
    
    return generator



In [ ]:
train_generator = get_train_generator(df = train_set,
                                      image_dir = None, 
                                      x_col = "FilePath",
                                      y_cols = labels, 
                                      batch_size=BATCH_SIZE,
                                      target_w = IMAGE_SIZE[0], 
                                      target_h = IMAGE_SIZE[1] 
                                      )

In [ ]:
#visualizing images
def get_label(y):
    ret_labels = []
    i = 0
    for idx in y:
        if idx:
            ret_labels.append(labels[i])
        i += 1
    if not ret_labels:
        return 'No Label'
    else:
        return '|'.join(ret_labels)

#get one batch of images from the imageset    
x, y = train_generator.__getitem__(0)



#show a set of images along with the labels appended at the top as title.
fig=plt.figure(figsize=(20, 10))
columns = 4; rows =2 
for i in tqdm(range(1, columns*rows +1)):
    fig.add_subplot(rows, columns, i)
    plt.imshow(x[i-1], cmap = 'gray')
    plt.title(get_label(y[i-1]))
    plt.axis(False)
    fig.add_subplot

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model


In [ ]:
with strategy.scope():
  
    def conv_block(x, growth_rate):
        x1 = BatchNormalization()(x)
        x1 = ReLU()(x1)
        x1 = Conv2D(filters=growth_rate, kernel_size=(3, 3), padding='same')(x1)
        x = Concatenate()([x, x1])
        return x

    def dense_block(x, num_layers, growth_rate):
        for _ in range(num_layers):
            x = conv_block(x, growth_rate)
        return x

    def transition_block(x, reduction):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(int(tf.keras.backend.int_shape(x)[-1] * reduction), kernel_size=(1, 1), padding='same')(x)
        x = tf.keras.layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
        return x

    def final_diseases_detection_model(input_shape=(224, 224, 3), num_classes=1000, growth_rate=32, num_blocks=[6, 12, 24, 16], reduction=0.5):
        inputs = Input(shape=input_shape)
        x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

        num_features = 64
        for i, num_layers in enumerate(num_blocks):
            x = dense_block(x, num_layers, growth_rate)
            num_features += num_layers * growth_rate
            if i != len(num_blocks) - 1:
                x = transition_block(x, reduction)

        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = GlobalAveragePooling2D()(x)
        x = Dense(num_classes, activation='softmax')(x)

        model = Model(inputs, x, name='final_diseases_detection_model')
        return model
    model = final_diseases_detection_model(input_shape = (224,224,3),num_classes=len(labels))
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint = ModelCheckpoint(
    'best_weights.h5',
    monitor='val_accuracy',  # Metric to monitor
    verbose=1,
    save_best_only=True,
    mode='max'  # Mode of the monitored metric (e.g., max for accuracy, min for loss)
)
    
    
model.compile(
    optimizer=tf.keras.optimizers.Adam( learning_rate=1e-4, amsgrad=False), 
    loss = 'binary_crossentropy',
#     loss = get_weighted_loss(pos_weights, neg_weights),
    metrics = ['binary_accuracy']
)

model.summary()



In [ ]:
checkpoint = ModelCheckpoint(
    'best_weights.h5',
    monitor='val_accuracy',  # Metric to monitor
    verbose=1,
    save_best_only=True,
    mode='max'  # Mode of the monitored metric (e.g., max for accuracy, min for loss)
)

In [ ]:
def build_lrfn(lr_start=0.002, lr_max=0.010, 
               lr_min=0, lr_rampup_epochs=8, 
               lr_sustain_epochs=0, lr_exp_decay=.8):

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

In [ ]:
history = model.fit_generator(train_generator, 
                              validation_data=valid_generator,
                              steps_per_epoch=len(train_generator), 
                              validation_steps=len(valid_generator), 
                              epochs = 30,
                              callbacks=[checkpoint,lr_schedule]
                             )

In [ ]:
def visualize_training(history, lw = 3):
    plt.figure(figsize=(10,6))
    plt.plot(history.history['binary_accuracy'], label = 'training', marker = '*', linewidth = lw)
    plt.plot(history.history['val_binary_accuracy'], label = 'validation', marker = 'o', linewidth = lw)
    plt.title('Training Accuracy vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(fontsize = 'x-large')
    plt.show()

    plt.figure(figsize=(10,6))
    plt.plot(history.history['loss'], label = 'training', marker = '*', linewidth = lw)
    plt.plot(history.history['val_loss'], label = 'validation', marker = 'o', linewidth = lw)
    plt.title('Training Loss vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(fontsize = 'x-large')
    plt.show()
visualize_training(history)  

In [ ]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator))
auc_rocs_before =get_roc_curve(labels, predicted_vals, test_generator, when = 'before training')

In [ ]:
model.save_weights('final_diseases_detection_model_weight.h5')

In [ ]:
train_df

In [ ]:
model.load_weights("final_diseases_detection_model_weight.h5")

In [ ]:
model = tf.keras.models.load_model("final_diseases_detection_model.h5")

In [ ]:
import cv2
import numpy as np

# Load your PNG image
image_path = '00009234_001.png'  # Replace with the path to your PNG image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure the image is in RGB format

# Resize the image to 224x224 to match the model's input shape
image = cv2.resize(image, (224, 224))

image = image / 255.0  # Normalize pixel values to [0, 1]

# Make a prediction using your model
prediction = model.predict(np.expand_dims(image, axis=0))  # Add an extra dimension for batch size

# Get the predicted class label and associated probability/confidence
predicted_label = np.argmax(prediction)
confidence = prediction[0][predicted_label]  # Probability of the predicted class

# If you want to map the class index to a class name
class_names = ["Atelectasis","Cardiomegaly","Consolidation","Edema","Effusion","Emphysema","Fibrosis","Hernia","Infiltration","Mass","Nodule","Pleural_Thickening","Pneumonia","Pneumothorax"]
predicted_class_name = class_names[predicted_label]

# Now 'predicted_class_name' contains the predicted class name for the single image,
# and 'confidence' contains the associated confidence.
print(f'Predicted Class: {predicted_class_name}')
print(f'Confidence: {confidence * 100:.2f}%')


In [ ]:
# Import the necessary modules
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
# Define the image size and the batch size
IMG_SIZE = 224
BATCH_SIZE = 32


In [ ]:
# Create a data generator that rescales the pixel values to [0, 1] range
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
# Load the images from a test directory and create a test dataset
test_ds = data_gen.flow_from_directory(
    directory="D:/TAHAB WORKING FOLDER/hi/images",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
# Load the model from the file my_model.h5
model = keras.models.load_model("final_diseases_detection_model.h5")

In [ ]:
results = model.evaluate(test_ds)
print("Test loss:", results[0])
print("Test accuracy:", results[1])